# Does hosting Olympics improve performance?

COMP 4462 Group 14

## Setup

In [89]:
import pandas as pd
import math

# Visualization
import altair as alt
from altair.expr import datum
from vega_datasets import data

## Load data and data preprocessing

### Load data

In [90]:
!pip -q install gdown

!gdown 'https://drive.google.com/uc?id=1RPTEYDYuENNC0Y_mUek-4vDwTM6H8ba1'
!unzip -d ./120_years_of_olympic_history 120_years_of_olympic_history.zip

!gdown 'https://drive.google.com/uc?id=1Sx81y0w5GhJMjMHZxs85iz9egZp96yfw'
!unzip -d ./2021_olympics_in_tokyo 2021_olympics_in_tokyo.zip

!gdown 'https://drive.google.com/uc?id=1veefAGwc5JjtmG1OegSnZQ73BhDTEKGx'
!unzip -d ./world_cities world_cities.zip

!gdown 'https://drive.google.com/uc?id=16rVCgGDC5MC3Cs0tZ4uwA7PV6FTSrpWn'
!unzip -d ./countries_codes countries_codes.zip

Downloading...
From: https://drive.google.com/uc?id=1RPTEYDYuENNC0Y_mUek-4vDwTM6H8ba1
To: /content/120_years_of_olympic_history.zip
100% 5.69M/5.69M [00:00<00:00, 23.0MB/s]
Archive:  120_years_of_olympic_history.zip
replace ./120_years_of_olympic_history/athlete_events.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./120_years_of_olympic_history/athlete_events.csv  
  inflating: ./120_years_of_olympic_history/noc_regions.csv  
Downloading...
From: https://drive.google.com/uc?id=1Sx81y0w5GhJMjMHZxs85iz9egZp96yfw
To: /content/2021_olympics_in_tokyo.zip
100% 356k/356k [00:00<00:00, 51.0MB/s]
Archive:  2021_olympics_in_tokyo.zip
replace ./2021_olympics_in_tokyo/Athletes.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./2021_olympics_in_tokyo/Athletes.xlsx  
  inflating: ./2021_olympics_in_tokyo/Coaches.xlsx  
  inflating: ./2021_olympics_in_tokyo/EntriesGender.xlsx  
  inflating: ./2021_olympics_in_tokyo/Medals.xlsx  
  inflating: ./2021_olympics_in_tokyo/Teams.xlsx  

In [91]:
athlete_events = pd.read_csv('./120_years_of_olympic_history/athlete_events.csv')
noc_regions = pd.read_csv('./120_years_of_olympic_history/noc_regions.csv')

medals_2021 = pd.read_excel('./2021_olympics_in_tokyo/Medals.xlsx')

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Data Preprocessing - Medals 2021

In [92]:
noc_regions

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [93]:
def mapping(col):
  if(col in set(noc_regions['region'])):
    results = noc_regions.loc[noc_regions['region'] == col, 'NOC'].values
    if (results.size == 1):
      return results[0]
    else:
      return 'MULTIPLE VALUE'
  else:
    return 'NOT FOUND'

In [94]:
# Convert ROC to RUS (Alternative code for Russia athletes)
medals_2021.at[medals_2021['Team/NOC'] == 'ROC', 'Team/NOC'] = 'RUS'

# medals_2021['Team/NOC'] contains both NOC and Country name
# Map all records to NOC only
medals_2021['NOC'] = medals_2021['Team/NOC'].apply(mapping)

In [95]:
# Manually change NOC in 2021 dataset (NOT FOUND)
missing_noc = {
    'United States of America': 'USA',
    'People\'s Republic of China': 'CHN',
    'Great Britain': 'GBR',
    'Republic of Korea': 'KOR',
    'Islamic Republic of Iran': 'IRI',
    'Chinese Taipei': 'TPE',
    'Hong Kong, China': 'HKG',
    'North Macedonia': 'MKD',
    'Syrian Arab Republic': 'SYR',
    'Republic of Moldova': 'MDA',
    'Côte d\'Ivoire': 'CIV',
    'RUS': 'RUS'
}

for country, noc in missing_noc.items():
  medals_2021.loc[(medals_2021['NOC'] == 'NOT FOUND') & (medals_2021['Team/NOC'] == country), 'NOC'] = noc

# Manually change NOC in 2021 dataset (MULTIPLE VALUE)
multiple_noc = {
    'Australia': 'AUS',
    'Germany': 'GER',
    'Canada': 'CAN',
    'Czech Republic': 'CZE',
    'Serbia': 'SRB',
    'Greece': 'GRE',
    'Malaysia': 'MAS'
}

for country, noc in multiple_noc.items():
  medals_2021.loc[(medals_2021['NOC'] == 'MULTIPLE VALUE') & (medals_2021['Team/NOC'] == country), 'NOC'] = noc

In [96]:
medals_2021

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total,NOC
0,1,United States of America,39,41,33,113,1,USA
1,2,People's Republic of China,38,32,18,88,2,CHN
2,3,Japan,27,14,17,58,5,JPN
3,4,Great Britain,22,21,22,65,4,GBR
4,5,RUS,20,28,23,71,3,RUS
...,...,...,...,...,...,...,...,...
88,86,Ghana,0,0,1,1,77,GHA
89,86,Grenada,0,0,1,1,77,GRN
90,86,Kuwait,0,0,1,1,77,KUW
91,86,Republic of Moldova,0,0,1,1,77,MDA


### Data Preprocessing - Athlete Events

In [97]:
athlete_events

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [98]:
# Records containing null values
athlete_events.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [99]:
# Extract athletes with medals
athletes_with_medals = athlete_events.dropna(subset=['Medal']) 

In [100]:
# Count number of medals
test = athletes_with_medals\
        .groupby(['Games', 'NOC', 'Medal'], as_index=False)[['ID']]\
        .count()\
        .rename(columns={'ID': 'count'})
test.head()

,Games,NOC,Medal,count
0,1896 Summer,AUS,Bronze,1
1,1896 Summer,AUS,Gold,2
2,1896 Summer,AUT,Bronze,2
3,1896 Summer,AUT,Gold,2
4,1896 Summer,AUT,Silver,1


In [101]:
# Convert table from long form to wide form
athletes_with_medals_wide = pd.pivot(test, index=['Games', 'NOC'], columns='Medal', values='count').reset_index()
athletes_with_medals_wide = athletes_with_medals_wide.fillna(0)
athletes_with_medals_wide['Total'] = athletes_with_medals_wide[['Bronze', 'Gold', 'Silver']].sum(axis=1)
athletes_with_medals_wide

Medal,Games,NOC,Bronze,Gold,Silver,Total
0,1896 Summer,AUS,1.0,2.0,0.0,3.0
1,1896 Summer,AUT,2.0,2.0,1.0,5.0
2,1896 Summer,DEN,3.0,1.0,2.0,6.0
3,1896 Summer,FRA,2.0,5.0,4.0,11.0
4,1896 Summer,GBR,3.0,3.0,3.0,9.0
...,...,...,...,...,...,...
1652,2016 Summer,UKR,5.0,2.0,8.0,15.0
1653,2016 Summer,USA,71.0,139.0,54.0,264.0
1654,2016 Summer,UZB,7.0,4.0,2.0,13.0
1655,2016 Summer,VEN,2.0,0.0,1.0,3.0


### Combine both datasets

In [102]:
# Combine athletes_with_medals_wide and medals_2021 together
medals_2021_modified = medals_2021.drop(['Rank', 'Rank by Total', 'Team/NOC'], axis=1)
medals_2021_modified['Games'] = '2021 Summer'

all_games_medal_results = pd.concat([athletes_with_medals_wide, medals_2021_modified])
all_games_medal_results

,Games,NOC,Bronze,Gold,Silver,Total
0,1896 Summer,AUS,1.0,2.0,0.0,3.0
1,1896 Summer,AUT,2.0,2.0,1.0,5.0
2,1896 Summer,DEN,3.0,1.0,2.0,6.0
3,1896 Summer,FRA,2.0,5.0,4.0,11.0
4,1896 Summer,GBR,3.0,3.0,3.0,9.0
...,...,...,...,...,...,...
88,2021 Summer,GHA,1.0,0.0,0.0,1.0
89,2021 Summer,GRN,1.0,0.0,0.0,1.0
90,2021 Summer,KUW,1.0,0.0,0.0,1.0
91,2021 Summer,MDA,1.0,0.0,0.0,1.0


In [103]:
# Export a CSV file for Tableau Task 1
all_games_medal_results = pd.merge(all_games_medal_results,noc_regions,on='NOC')
all_games_medal_results = all_games_medal_results[all_games_medal_results['region']!='Individual Olympic Athletes']
all_games_medal_results['Seasons'] = all_games_medal_results['Games'].str[5:]
all_games_medal_results.to_csv("all_games_medal_results.csv", index=False)

In [104]:
# Map the NOC back to Countries name
def noc_to_country(col):
  if(col in set(noc_regions['NOC'])):
    results = noc_regions.loc[noc_regions['NOC'] == col, 'region'].values
    if (results.size == 1):
      return results[0]
    else:
      return 'MULTIPLE'
  else:
    return "NOT FOUND"

In [105]:
# Map the NOC back to Countries name
all_games_medal_results['country'] = all_games_medal_results['NOC'].apply(noc_to_country)

# Manually fix the NOC that is NOT FOUND (i.e. SGP to Singapore)
all_games_medal_results.loc[(all_games_medal_results['country'] == 'NOT FOUND') & (all_games_medal_results['NOC'] == 'SGP'), 'country'] = 'Singapore'

# Convert NOC (i.e. IOC) code to ISO 3166-1
countries_codes = pd.read_excel('./countries_codes/countriesCodes.xlsx')

def ioc_to_iso3166(col):
  if(col in set(countries_codes['IOC'])):
    return countries_codes.loc[countries_codes['IOC'] == col, 'ISO 3166-1 numeric'].values[0]
  else:
    return 'NOT FOUND'

all_games_medal_results['iso_3166_numeric'] = all_games_medal_results['NOC'].apply(ioc_to_iso3166)

In [106]:
# Fix the NOC that is NOT FOUND
noc_not_found = {
    'Czech Republic': 203,
    'Australia': 36,
    'Serbia': 688,
    'Lebanon': 422,
    'Russia': 643,
    'Syria': 760,
    'Trinidad': 780,
    'Germany': 276,
    'Curacao': 531
}

for country, iso in noc_not_found.items():
  all_games_medal_results.loc[(all_games_medal_results['country'] == country) & (all_games_medal_results['iso_3166_numeric'] == 'NOT FOUND'), 'iso_3166_numeric'] = iso

In [107]:
all_games_medal_results

,Games,NOC,Bronze,Gold,Silver,Total,region,notes,Seasons,country,iso_3166_numeric
0,1896 Summer,AUS,1.0,2.0,0.0,3.0,Australia,NaN,Summer,Australia,36.0
1,1900 Summer,AUS,3.0,3.0,0.0,6.0,Australia,NaN,Summer,Australia,36.0
2,1904 Summer,AUS,1.0,0.0,3.0,4.0,Australia,NaN,Summer,Australia,36.0
3,1906 Summer,AUS,3.0,0.0,0.0,3.0,Australia,NaN,Summer,Australia,36.0
4,1920 Summer,AUS,1.0,0.0,6.0,7.0,Australia,NaN,Summer,Australia,36.0
...,...,...,...,...,...,...,...,...,...,...,...
1741,2016 Summer,KOS,0.0,1.0,0.0,1.0,Kosovo,NaN,Summer,Kosovo,NaN
1742,2021 Summer,KOS,0.0,2.0,0.0,2.0,Kosovo,NaN,Summer,Kosovo,NaN
1743,2021 Summer,SMR,2.0,0.0,1.0,3.0,San Marino,NaN,Summer,San Marino,674.0
1744,2021 Summer,TKM,0.0,0.0,1.0,1.0,Turkmenistan,NaN,Summer,Turkmenistan,795.0


### Create another df containing participants number (before Tokyo 2021)

In [108]:
athlete_events_with_participated = athlete_events
athlete_events_with_participated['Medal'] = athlete_events_with_participated['Medal'].fillna(value='No Medal')

athlete_events_with_participated_grouped = athlete_events_with_participated\
        .groupby(['Games', 'NOC', 'Medal'], as_index=False)[['ID']]\
        .count()\
        .rename(columns={'ID': 'count'})
athlete_events_with_participated_grouped.head()

,Games,NOC,Medal,count
0,1896 Summer,AUS,Bronze,1
1,1896 Summer,AUS,Gold,2
2,1896 Summer,AUS,No Medal,2
3,1896 Summer,AUT,Bronze,2
4,1896 Summer,AUT,Gold,2


In [109]:
athletes_with_participated_wide = pd.pivot(athlete_events_with_participated_grouped, index=['Games', 'NOC'], columns='Medal', values='count').reset_index()
athletes_with_participated_wide = athletes_with_participated_wide.fillna(0)
athletes_with_participated_wide['Total Medal'] = athletes_with_participated_wide[['Bronze', 'Gold', 'Silver']].sum(axis=1)
athletes_with_participated_wide['Total Athletes'] = athletes_with_participated_wide[['Total Medal', 'No Medal']].sum(axis=1)
athletes_with_participated_wide

Medal,Games,NOC,Bronze,Gold,No Medal,Silver,Total Medal,Total Athletes
0,1896 Summer,AUS,1.0,2.0,2.0,0.0,3.0,5.0
1,1896 Summer,AUT,2.0,2.0,3.0,1.0,5.0,8.0
2,1896 Summer,DEN,3.0,1.0,9.0,2.0,6.0,15.0
3,1896 Summer,FRA,2.0,5.0,15.0,4.0,11.0,26.0
4,1896 Summer,GBR,3.0,3.0,16.0,3.0,9.0,25.0
...,...,...,...,...,...,...,...,...
3832,2016 Summer,VIE,0.0,1.0,26.0,1.0,2.0,28.0
3833,2016 Summer,VIN,0.0,0.0,4.0,0.0,0.0,4.0
3834,2016 Summer,YEM,0.0,0.0,3.0,0.0,0.0,3.0
3835,2016 Summer,ZAM,0.0,0.0,7.0,0.0,0.0,7.0


In [110]:
# Map the NOC back to Countries name
athletes_with_participated_wide['country'] = athletes_with_participated_wide['NOC'].apply(noc_to_country)

# Manually fix the NOC that is NOT FOUND (i.e. SGP to Singapore)
athletes_with_participated_wide.loc[(athletes_with_participated_wide['country'] == 'NOT FOUND') & (athletes_with_participated_wide['NOC'] == 'SGP'), 'country'] = 'Singapore'

# Convert NOC (i.e. IOC) code to ISO 3166-1
countries_codes = pd.read_excel('./countries_codes/countriesCodes.xlsx')

def ioc_to_iso3166(col):
  if(col in set(countries_codes['IOC'])):
    return countries_codes.loc[countries_codes['IOC'] == col, 'ISO 3166-1 numeric'].values[0]
  else:
    return 'NOT FOUND'

athletes_with_participated_wide['iso_3166_numeric'] = athletes_with_participated_wide['NOC'].apply(ioc_to_iso3166)

In [111]:
for country, iso in noc_not_found.items():
  athletes_with_participated_wide.loc[
      (athletes_with_participated_wide['country'] == country) 
        & (athletes_with_participated_wide['iso_3166_numeric'] == 'NOT FOUND'), 
      'iso_3166_numeric'
] = iso

In [112]:
athletes_with_participated_wide

Medal,Games,NOC,Bronze,Gold,No Medal,Silver,Total Medal,Total Athletes,country,iso_3166_numeric
0,1896 Summer,AUS,1.0,2.0,2.0,0.0,3.0,5.0,Australia,36.0
1,1896 Summer,AUT,2.0,2.0,3.0,1.0,5.0,8.0,Austria,40.0
2,1896 Summer,DEN,3.0,1.0,9.0,2.0,6.0,15.0,Denmark,208.0
3,1896 Summer,FRA,2.0,5.0,15.0,4.0,11.0,26.0,France,250.0
4,1896 Summer,GBR,3.0,3.0,16.0,3.0,9.0,25.0,UK,826.0
...,...,...,...,...,...,...,...,...,...,...
3832,2016 Summer,VIE,0.0,1.0,26.0,1.0,2.0,28.0,Vietnam,704.0
3833,2016 Summer,VIN,0.0,0.0,4.0,0.0,0.0,4.0,Saint Vincent,670.0
3834,2016 Summer,YEM,0.0,0.0,3.0,0.0,0.0,3.0,Yemen,887.0
3835,2016 Summer,ZAM,0.0,0.0,7.0,0.0,0.0,7.0,Zambia,894.0


## Task 1 - Summarize the total medal by country for summer and winter separately

### Map Visualization Implementation

In [113]:
# Group the result for all Summer Olympics
task1_summer_df = all_games_medal_results[all_games_medal_results['Games'].str.contains('Summer')]
task1_summer_df = task1_summer_df.groupby(['country', 'iso_3166_numeric'], as_index=False).sum()

# # Get and add the countries that is not in task1_summer_df (i.e. #medals == 0)
# missing_t1_s = countries_codes.merge(
#     task1_summer_df,
#     left_on='ISO 3166-1 numeric',
#     right_on='iso_3166_numeric',
#     how='left',
#     indicator=True
# )
# missing_t1_s = missing_t1_s[missing_t1_s['_merge'] == 'left_only']
# missing_t1_s = missing_t1_s[['Country', 'ISO 3166-1 numeric']]
# missing_t1_s['Bronze'], missing_t1_s['Gold'], missing_t1_s['Silver'], missing_t1_s['Total'] = [0,0,0,0]
# missing_t1_s = missing_t1_s.rename(columns={'Country': 'country', 'ISO 3166-1 numeric': 'iso_3166_numeric'})
# task1_summer_df = pd.concat([task1_summer_df, missing_t1_s])

# Group the result for all Winter Olympics
task1_winter_df = all_games_medal_results[all_games_medal_results['Games'].str.contains('Winter')]
task1_winter_df = task1_winter_df.groupby(['country', 'iso_3166_numeric'], as_index=False).sum()

# # Get and add the countries that is not in task1_winter_df (i.e. #medals == 0)
# missing_t1_w = countries_codes.merge(
#     task1_winter_df,
#     left_on='ISO 3166-1 numeric',
#     right_on='iso_3166_numeric',
#     how='left',
#     indicator=True
# )
# missing_t1_w = missing_t1_w[missing_t1_w['_merge'] == 'left_only']
# missing_t1_w = missing_t1_w[['Country', 'ISO 3166-1 numeric']]
# missing_t1_w['Bronze'], missing_t1_w['Gold'], missing_t1_w['Silver'], missing_t1_w['Total'] = [0,0,0,0]
# missing_t1_w = missing_t1_w.rename(columns={'Country': 'country', 'ISO 3166-1 numeric': 'iso_3166_numeric'})
# task1_winter_df = pd.concat([task1_winter_df, missing_t1_w])

In [114]:
task1_summer_df

,country,iso_3166_numeric,Bronze,Gold,Silver,Total
0,Afghanistan,4.0,2.0,0.0,0.0,2.0
1,Algeria,12.0,8.0,5.0,4.0,17.0
2,Argentina,32.0,93.0,91.0,93.0,277.0
3,Armenia,51.0,11.0,2.0,7.0,20.0
4,Australia,36.0,537.0,379.0,463.0,1379.0
...,...,...,...,...,...,...
131,Venezuela,862.0,10.0,3.0,6.0,19.0
132,Vietnam,704.0,0.0,1.0,3.0,4.0
133,"Virgin Islands, US",850.0,0.0,0.0,1.0,1.0
134,Zambia,894.0,1.0,0.0,1.0,2.0


In [115]:
countries = alt.topo_feature(data.world_110m.url, 'countries')

task1_summer_map = alt.Chart(countries).mark_geoshape(
    stroke='black'
).encode(
    color=alt.Color('Total:Q',
                    title='Total Medal Obtained',
                    # scale=alt.Scale(
                    #     scheme='greens',
                    #     domain=[1, task1_summer_df['Total'].max()]
                    # )
                    ),
    tooltip=['country:N', 'Total:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(task1_summer_df, 'iso_3166_numeric', ['country', 'Total'])
).properties(
    projection={'type': 'equirectangular'},
    width=900,
    height=600,
    title='Total medals obtained by each countries in Summer Olympics from 1896 to 2020'
)

task1_winter_map = alt.Chart(countries).mark_geoshape(
    stroke='black'
).encode(
    color=alt.Color('Total:Q',
                    title='Total Medal Obtained',
                    # scale=alt.Scale(scheme='greens')
                    ),
    tooltip=['country:N', 'Total:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(task1_winter_df, 'iso_3166_numeric', ['country', 'Total'])
).properties(
    projection={'type': 'equirectangular'},
    width=900,
    height=600,
    title='Total medals obtained by each countries in Winter Olympics from 1896 to 2020'
)

## Task 2 - Compare the number of medals by country for each game with the hosting country highlighted to see the hosting country's rank

### Setup - create hosting country list

In [116]:
def host_country(city):
    city_dict = {"Barcelona": "Spain", "Rio de Janeiro": "Brazil", "London": "United Kingdom", "Beijing": "China",
                "Athina": "Greece", "Sydney": "Australia", "Melbourne": "Australia", "Atlanta": "United States",
                "Los Angeles": "United States", "St. Louis": "United States", "Barcelona": "Spain", 
                "Seoul": "South Korea", "Moskva": "Russia", "Montreal": "Canada", "Munich": "Germany", "Berlin": "Germany",
                "Mexico City": "Mexico", "Tokyo": "Japan", "Roma": "Italy", "Paris": "France", "Helsinki": "Finland", 
                "Amsterdam": "Netherlands", "Antwerpen": "Belgium", "Stockholm": "Sweden", "Calgary": "Canada", 
                "Chamonix": "France", "Sankt Moritz": "Switzerland", "Lake Placid": "United States",
                 "Garmisch-Partenkirchen": "Germany", "Oslo": "Norway", "Cortina d'Ampezzo": "Italy", 
                 "Squaw Valley": "United States", "Innsbruck": "Austria", "Grenoble": "France",
                 "Sapporo": "Japan", "Sarajevo": "Yugoslavia", "Albertville": "France", "Lillehammer": "Norway",
                 "Nagano": "Japan", "Salt Lake City": "United States", "Torino": "Italy",
                 "Vancouver": "Canada", "Sochi": "Russia"}

    if city in city_dict:
        return city_dict[city]
    else:
        return "Other"

athlete_events['Hosting_country'] = athlete_events['City'].apply(host_country)

In [117]:
hosting_city = athlete_events.drop_duplicates(['Games', 'City'])[['Games', 'City']]
hosting_city = hosting_city.append({
    'City': 'Tokyo',
    'Games': '2021 Summer'
}, ignore_index=True)

In [118]:
def country_to_iso(col):
  if(col in set(countries_codes['Country'])):
    return countries_codes.loc[countries_codes['Country'] == col, 'ISO 3166-1 numeric'].values[0]
  else:
    return 'NOT FOUND'

In [119]:
hosting_city['Country'] = hosting_city['City'].apply(host_country)
hosting_city['iso_3166_numeric'] = hosting_city['Country'].apply(country_to_iso)
hosting_city

,Games,City,Country,iso_3166_numeric
0,1992 Summer,Barcelona,Spain,724.0
1,2012 Summer,London,United Kingdom,826.0
2,1920 Summer,Antwerpen,Belgium,56.0
3,1900 Summer,Paris,France,250.0
4,1988 Winter,Calgary,Canada,124.0
5,1992 Winter,Albertville,France,250.0
6,1994 Winter,Lillehammer,Norway,578.0
7,1932 Summer,Los Angeles,United States,840.0
8,2002 Winter,Salt Lake City,United States,840.0
9,1952 Summer,Helsinki,Finland,246.0


### Implementation (Map)

In [120]:
# A dropdown filter
games = all_games_medal_results['Games'].drop_duplicates().values
games_dropdown = alt.binding_select(options=games)
games_select = alt.selection_single(fields=['Games'], bind=games_dropdown, name="games", init={'Games': '2021 Summer'})

task2_map = alt.Chart(all_games_medal_results).mark_geoshape(
    stroke='black'
).encode(
    color=alt.Color('Total:Q',
                    title='Total Medal Obtained',
                    scale=alt.Scale(scheme='greens')),
    tooltip=['country:N', 'Total:Q']
).add_selection(
    games_select
).transform_filter(
    games_select
).transform_lookup(
    lookup='iso_3166_numeric',
    from_=alt.LookupData(countries, key='id', fields=['type', 'properties', 'geometry'])
).properties(
    width=900,
    height=600,
    title='Total medals obtained by each countries in Olympic'
)

map_hosting_highlight = alt.Chart(hosting_city).mark_geoshape(
    fill='transparent',
    stroke='red',
    strokeWidth=5
).transform_filter(
    games_select
).transform_lookup(
    lookup='iso_3166_numeric',
    from_=alt.LookupData(countries, key='id', fields=['type', 'properties', 'geometry'])
)

### Implementation (Scatter Plot)

In [121]:
# Add hosting bool to dataframe

task2_scatter_df = all_games_medal_results
task2_scatter_df = task2_scatter_df.merge(
    hosting_city,
    on=['Games', 'iso_3166_numeric'],
    how='left',
    indicator=True
)
task2_scatter_df['hosting'] = (task2_scatter_df['_merge'] == 'both')
task2_scatter_df = task2_scatter_df[['Games', 'NOC', 'Bronze', 'Gold', 'Silver', 'Total', 'country', 'iso_3166_numeric', 'hosting']]
task2_scatter_df

# # Filter record with year >= 2000
# task2_scatter_demo_df = task2_scatter_df
# task2_scatter_demo_df[['year', 'season']] = task2_scatter_demo_df['Games'].str.split(' ', expand=True)
# task2_scatter_demo_df['year'] = task2_scatter_demo_df['year'].astype(int)
# task2_scatter_demo_df = task2_scatter_demo_df[task2_scatter_demo_df['year'] >= 2000]

,Games,NOC,Bronze,Gold,Silver,Total,country,iso_3166_numeric,hosting
0,1896 Summer,AUS,1.0,2.0,0.0,3.0,Australia,36.0,False
1,1900 Summer,AUS,3.0,3.0,0.0,6.0,Australia,36.0,False
2,1904 Summer,AUS,1.0,0.0,3.0,4.0,Australia,36.0,False
3,1906 Summer,AUS,3.0,0.0,0.0,3.0,Australia,36.0,False
4,1920 Summer,AUS,1.0,0.0,6.0,7.0,Australia,36.0,False
...,...,...,...,...,...,...,...,...,...
1739,2016 Summer,KOS,0.0,1.0,0.0,1.0,Kosovo,NaN,False
1740,2021 Summer,KOS,0.0,2.0,0.0,2.0,Kosovo,NaN,False
1741,2021 Summer,SMR,2.0,0.0,1.0,3.0,San Marino,674.0,False
1742,2021 Summer,TKM,0.0,0.0,1.0,1.0,Turkmenistan,795.0,False


In [122]:
task2_scatter = alt.Chart(task2_scatter_df).mark_point().encode(
    x='Games',
    y='Total',
    color='hosting',
    tooltip=['country', 'Total']
).properties(
    width=700,
    height=400,
    title='Total medals obtained by each countries by Olympics games'
).interactive()

## Task 3 - Compare the number of medals in the hosting year with non-hosting year

In [123]:
# Add hosting bool to dataframe
task3_df = all_games_medal_results
task3_df = task3_df.merge(
    hosting_city,
    on=['Games', 'iso_3166_numeric'],
    how='left',
    indicator=True
)
task3_df['hosting'] = (task3_df['_merge'] == 'both')
task3_df = task3_df[['Games', 'NOC', 'Bronze', 'Gold', 'Silver', 'Total', 'country', 'iso_3166_numeric', 'hosting']]

# Keep records only if the country has been hosting country once
hosting_country_iso_3166 = list(hosting_city[hosting_city['iso_3166_numeric'] != 'NOT FOUND']['iso_3166_numeric'])
task3_df = task3_df[task3_df['iso_3166_numeric'].isin(hosting_country_iso_3166)]
task3_df

# # Filter record with year >= 2000
# task3_demo_df = task3_df
# task3_demo_df[['year', 'season']] = task3_demo_df['Games'].str.split(' ', expand=True)
# task3_demo_df['year'] = task3_demo_df['year'].astype(int)
# task3_demo_df = task3_demo_df[task3_demo_df['year'] >= 2000]

,Games,NOC,Bronze,Gold,Silver,Total,country,iso_3166_numeric,hosting
0,1896 Summer,AUS,1.0,2.0,0.0,3.0,Australia,36.0,False
1,1900 Summer,AUS,3.0,3.0,0.0,6.0,Australia,36.0,False
2,1904 Summer,AUS,1.0,0.0,3.0,4.0,Australia,36.0,False
3,1906 Summer,AUS,3.0,0.0,0.0,3.0,Australia,36.0,False
4,1920 Summer,AUS,1.0,0.0,6.0,7.0,Australia,36.0,False
...,...,...,...,...,...,...,...,...,...
1496,2014 Winter,CHN,5.0,3.0,4.0,12.0,China,156.0,False
1497,2016 Summer,CHN,37.0,46.0,30.0,113.0,China,156.0,False
1498,2021 Summer,CHN,18.0,38.0,32.0,88.0,China,156.0,False
1544,1992 Summer,EUN,67.0,92.0,61.0,220.0,Russia,643,False


In [124]:
task3_result = alt.Chart(task3_df).mark_point().encode(
    x='Total',
    y='country',
    color='hosting',
    tooltip=['Games', 'Total']
).properties(
    width=700,
    height=400,
    title='Total Number of Medal for Different Countries in each Hosting City'
).interactive()

## Task 4 - Explore correlation between number of medals, number of participants, and whether the country is hosting

In [125]:
# Add hosting bool to dataframe

task4 = athletes_with_participated_wide
task4 = task4.merge(
    hosting_city,
    on=['Games', 'iso_3166_numeric'],
    how='left',
    indicator=True
)
task4['hosting'] = (task4['_merge'] == 'both')
task4 = task4[['Games', 'NOC', 'Bronze', 'Gold', 'Silver', 'Total Medal', 'Total Athletes', 'country', 'iso_3166_numeric', 'hosting']]
task4

,Games,NOC,Bronze,Gold,Silver,Total Medal,Total Athletes,country,iso_3166_numeric,hosting
0,1896 Summer,AUS,1.0,2.0,0.0,3.0,5.0,Australia,36.0,False
1,1896 Summer,AUT,2.0,2.0,1.0,5.0,8.0,Austria,40.0,False
2,1896 Summer,DEN,3.0,1.0,2.0,6.0,15.0,Denmark,208.0,False
3,1896 Summer,FRA,2.0,5.0,4.0,11.0,26.0,France,250.0,False
4,1896 Summer,GBR,3.0,3.0,3.0,9.0,25.0,UK,826.0,False
...,...,...,...,...,...,...,...,...,...,...
3832,2016 Summer,VIE,0.0,1.0,1.0,2.0,28.0,Vietnam,704.0,False
3833,2016 Summer,VIN,0.0,0.0,0.0,0.0,4.0,Saint Vincent,670.0,False
3834,2016 Summer,YEM,0.0,0.0,0.0,0.0,3.0,Yemen,887.0,False
3835,2016 Summer,ZAM,0.0,0.0,0.0,0.0,7.0,Zambia,894.0,False


In [126]:
source = task4

base = alt.Chart(source).transform_window(
    index='count()'
).transform_fold(
    ['hosting', 'Total Medal', 'Total Athletes']
).transform_joinaggregate(
    min="min(value)",
    max="max(value)",
    groupby=["key"]
).transform_calculate(
    norm_val="(datum.value - datum.min) / (datum.max - datum.min)",
    mid="(datum.min + datum.max) / 2"
).properties(
    width=900,
    height=600,
    title='Correlation between #medals, #participants, and hosting'
)

lines = base.mark_line(opacity=0.2).encode(
    x='key:N',
    y=alt.Y('norm_val:Q', axis=None),
    detail="index:N"
)

rules = base.mark_rule(
    color="#ccc", tooltip=None
).encode(
    x="key:N",
    detail="count():Q",
)

def ytick(yvalue, field):
    scale = base.encode(x='key:N', y=alt.value(yvalue), text=f"min({field}):Q")
    return alt.layer(
        scale.mark_text(baseline="middle", align="right", dx=-5, tooltip=None),
        scale.mark_tick(size=8, color="#ccc", orient="horizontal", tooltip=None)
    )

task4_result = alt.layer(
    lines, rules, ytick(0, "max"), ytick(300, "mid"), ytick(600, "min")
).configure_axisX(
    domain=False, labelAngle=0, tickColor="#ccc", title=None
).configure_view(
    stroke=None
)

## Results

### Task 1

In [128]:
task1_summer_map.display()
task1_winter_map.display()

alt.Chart(...)

alt.Chart(...)

### Task 2

In [129]:
map_hosting_highlight + task2_map

alt.LayerChart(...)

In [130]:
task2_scatter

alt.Chart(...)

### Task 3

In [131]:
task3_result

alt.Chart(...)

### Task 4

In [132]:
task4_result

alt.LayerChart(...)